In [2]:
# Import functions from modules
%reload_ext autoreload
%autoreload 2
from pretreatment_methods import *
from w2v_feats_adding import *

Using TensorFlow backend.
2017-12-11 02:36:07,066 : INFO : 'pattern' package not found; tag filters are not available for English


In [3]:
pre_local="/home/jlevyabi/seacabo/fempl_big_sim/sent_class/twitter-datasets/"
pre_remote="/datastore/complexnet/jlevyabi/network_representation/data_files/w2v_files/word2vec_twitter_model/"
train_data=TwitterSentimentData(loc_data=pre_local,pre_cleaned=False,is_test_file=False)
test_data=TwitterSentimentData(loc_data=pre_local,pre_cleaned=False,is_test_file=True)

In [4]:
# Clean Data
complete_data=TwitterAddTextData(train_data,1,1,1,1,1,1,1)
complete_data.add_text_features()
cleaned=TwitterDataCleaning(complete_data)
sentiment_data=SentimentAnalysis(cleaned)

In [5]:
# Analysis Data
sentiment_data.polar_df_3grams[(sentiment_data.polar_df_3grams.polarity<0.1)
                               & (sentiment_data.polar_df_3grams.total>100 )].sample(10)

,3gram,pos_sent,neg_sent,total,polarity
746003,"((, hardcover, the)",0,116,116,0.00000
748268,"(,, -, ounce)",0,371,371,0.00000
747857,"(is, written, in)",0,128,128,0.00000
746671,"(mb, ddr, -)",0,132,132,0.00000
870812,"(reliever, &, anti-depressant)",0,119,119,0.00000
1063100,"(., ->, .)",0,142,142,0.00000
85378,"(i, feel, so)",16,149,165,0.09697
744637,"(brand, gb, ddr)",0,193,193,0.00000
743160,"((, mass, market)",0,240,240,0.00000
870813,"(&, anti-depressant, in)",0,119,119,0.00000


In [6]:
full_data=Tweets_w2v_Features(pre_filled_feats=cleaned.info,built_w2v=pre_remote+"word2vec_twitter_model.bin")

2017-12-11 02:53:09,866 : INFO : loading projection weights from /datastore/complexnet/jlevyabi/network_representation/data_files/w2v_files/word2vec_twitter_model/word2vec_twitter_model.bin
2017-12-11 02:59:04,396 : INFO : loaded (3039345, 400) matrix from /datastore/complexnet/jlevyabi/network_representation/data_files/w2v_files/word2vec_twitter_model/word2vec_twitter_model.bin


In [7]:
full_data.sp_features(min_times=1)

Nb words in vocabulary.... 38365
Computing similarity matrix .....


100%|██████████| 35946/35946 [00:02<00:00, 15975.29it/s]


Spectral Clustering ....
SC labeling .... 


In [8]:
full_data.nlm_features()

In [9]:
def get_embedding(embed_name,df):
    dim_total=len((df.head(1)[embed_name][0]))
    return np.array(np.vstack([sample[0]
                      for it,sample in (df[[embed_name]].iterrows())]))

w2v_fts=get_embedding("w2v",full_data.info.data[full_data.info.data['w2v'].isnull()== False])
cfd_fts=get_embedding("cfd",full_data.info.data[full_data.info.data['w2v'].isnull()== False])
shallow=["number_of_uppercases","number_of_exclamations","number_of_questions","number_of_sights",
         "number_of_hashtags","number_of_mentions","number_of_urls"]
shallow_fts=full_data.info.data[full_data.info.data['w2v'].isnull()== False][shallow]

In [10]:
target=full_data.info.data[full_data.info.data['w2v'].isnull()== False]["pos_sent"]
fts=np.hstack([w2v_fts,cfd_fts,shallow_fts])

In [15]:
all_but_text=full_data.info.data.drop("text",axis=1)
import pickle
pickle.dump(all_but_text,open("/home/jlevyabi/seacabo/fempl_big_sim/sent_class/checkpt_data/dic_all.p","wb"))

#all_but_text=pickle.load(open("/home/jlevyabi/seacabo/fempl_big_sim/sent_class/checkpt_data/dic_all.p","rb"))

w2v_fts=get_embedding("w2v",all_but_text[all_but_text['w2v'].isnull()== False])
cfd_fts=get_embedding("cfd",all_but_text[all_but_text['w2v'].isnull()== False])
shallow=["number_of_uppercases","number_of_exclamations","number_of_questions","number_of_sights",
         "number_of_hashtags","number_of_mentions","number_of_urls"]
shallow_fts=all_but_text[all_but_text['w2v'].isnull()== False][shallow]

target=all_but_text[all_but_text['w2v'].isnull()== False]["pos_sent"]
fts=np.hstack([w2v_fts,cfd_fts,shallow_fts])

In [41]:
from sklearn.model_selection import KFold, train_test_split
target_two_cols=np.transpose(np.vstack([target,target]))
target_two_cols[target_two_cols[:,0]==-1,0]=0
target_two_cols[target_two_cols[:,1]==1,1]=0
target_two_cols=np.abs(target_two_cols)
X_train, X_test, y_train, y_test=train_test_split(fts,target_two_cols)

In [18]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression(C=1e4,verbose=5)
logreg.fit(X_train, y_train)
pred=logreg.predict(X_test)print(classification_report(y_test, pred))

             precision    recall  f1-score   support

         -1       0.83      0.79      0.81     24738
          1       0.80      0.83      0.82     24497

avg / total       0.81      0.81      0.81     49235



In [ ]:
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, RMSprop, Nadam, Adagrad, Adadelta, Adam, Adamax

def build_model(input_dim, nb_class):
    units=[input_dim//2,input_dim//4,input_dim//4,input_dim//10,input_dim//50]
    model = Sequential()
    model.add(Dense(input_dim=input_dim, units=units[0], activation='relu'))
    model.add(Dropout(0.7))
    model.add(Dense(input_dim=units[0], units=units[1], activation='relu'))
    model.add(Dropout(0.8))
    model.add(Dense(input_dim=units[1], units=units[2], activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(input_dim=units[2], units=units[3], activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(input_dim=units[3], units=units[4], activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(output_dim=nb_class, activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer='adadelta')
    return model

checkpointer = ModelCheckpoint(filepath='/home/jlevyabi/seacabo/fempl_big_sim/sent_class/checkpt_data/weights.hdf5',
                               verbose=1, save_best_only=True)
tf_model=build_model(input_dim=X_train.shape[1],nb_class=2)
tf_model.fit(X_train, y_train, batch_size=1000, epochs=1000, verbose=2)
score = tf_model.evaluate(X_test, y_test, batch_size=1000)

preds=tf_model.predict(X_test)
predis=np.int64(preds>0.5)
print(classification_report(y_test[:,0],predis[:,0]))
             precision    recall  f1-score   support

          0       0.85      0.83      0.84     24800
          1       0.83      0.85      0.84     24435

avg / total       0.84      0.84      0.84     49235